In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
# Define data directories
train_dir = r'dataset/split_ttv_dataset_type_of_plants/Train_Set_Folder'
test_dir = r'dataset/split_ttv_dataset_type_of_plants/Test_Set_Folder'

# Define image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

# Create data generators for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,  # Normalize pixel values to [0, 1]
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 23972 images belonging to 30 classes.
Found 2998 images belonging to 30 classes.


In [5]:
# Create the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_generator.class_indices), activation='softmax'))

In [6]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
epochs = 6  # You can adjust this number based on your needs
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

Epoch 1/6
750/750 [==============================] - 1123s 1s/step - loss: 2.6863 - accuracy: 0.2021 - val_loss: 2.0146 - val_accuracy: 0.3766
Epoch 2/6
750/750 [==============================] - 1065s 1s/step - loss: 2.0936 - accuracy: 0.3584 - val_loss: 1.8013 - val_accuracy: 0.4466
Epoch 3/6
750/750 [==============================] - 1122s 1s/step - loss: 1.8536 - accuracy: 0.4299 - val_loss: 1.5499 - val_accuracy: 0.5147
Epoch 4/6
750/750 [==============================] - 1037s 1s/step - loss: 1.7034 - accuracy: 0.4713 - val_loss: 1.4146 - val_accuracy: 0.5597
Epoch 5/6
750/750 [==============================] - 1052s 1s/step - loss: 1.5817 - accuracy: 0.5063 - val_loss: 1.2826 - val_accuracy: 0.6037
Epoch 6/6
750/750 [==============================] - 999s 1s/step - loss: 1.4882 - accuracy: 0.5358 - val_loss: 1.2277 - val_accuracy: 0.6097


In [1]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print("Test accuracy:", test_acc)

NameError: name 'model' is not defined

In [9]:
# Save the model
model.save('plant_type_classifier.h5')